In [20]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec


In [21]:
pc=Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag", dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '83cc92b5cae63b724a7ff34188b4fcc3', 'Date': 'Sat, 24 Aug 2024 01:32:25 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [ ]:
import json
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. John Smith',
  'subject': 'Computer Science',
  'stars': 5,
  'review': "Dr. Smith's lectures are clear and concise. He makes complex topics easy to understand."},
 {'professor': 'Dr. Emily Johnson',
  'subject': 'Mathematics',
  'stars': 4,
  'review': 'Dr. Johnson is very knowledgeable, but sometimes the pace is a bit fast.'},
 {'professor': 'Dr. Michael Brown',
  'subject': 'Physics',
  'stars': 3,
  'review': 'The lectures are informative, but the exams are tough. Attend all classes!'},
 {'professor': 'Dr. Sarah Davis',
  'subject': 'Chemistry',
  'stars': 5,
  'review': 'Dr. Davis is enthusiastic and makes the subject enjoyable. Highly recommended!'},
 {'professor': 'Dr. William Miller',
  'subject': 'History',
  'stars': 2,
  'review': "The material is interesting, but Dr. Miller's lectures are a bit dry."},
 {'professor': 'Dr. Linda Wilson',
  'subject': 'Biology',
  'stars': 4,
  'review': 'Dr. Wilson is great at explaining difficult concepts, but there’s a 

In [27]:
processed_data = []
client = OpenAI(api_key=os.getenv("OPEN_AI_API_KEY"))

for review in data['reviews']:
    response = client.embeddings.create(
        input = review['review'],
        model="text-embedding-3-small",
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"]
        }
    })

In [28]:
processed_data[0]

{'values': [-0.017215507,
  0.010556678,
  -0.03857248,
  -0.0009389354,
  0.018974952,
  0.00076172187,
  -0.007829537,
  0.024767593,
  0.040061243,
  -0.04620577,
  0.028286485,
  -0.0197058,
  -0.016565865,
  0.008025783,
  0.04011538,
  -0.009528079,
  -0.034268603,
  -0.008086687,
  0.02920681,
  0.045123033,
  0.053081147,
  -0.0026137931,
  0.03708372,
  -0.008567151,
  -0.04571854,
  -0.027122544,
  0.026459368,
  0.032184336,
  0.013967298,
  0.013838723,
  0.05559851,
  -0.012843959,
  0.0006796708,
  0.0072881687,
  -0.024902934,
  0.02054492,
  -0.03196779,
  0.0065336367,
  -0.011240156,
  -0.005027957,
  0.042389125,
  0.0030976408,
  -0.053135283,
  -0.027934596,
  0.0533789,
  -0.015659073,
  -0.017201973,
  -0.0056133107,
  0.022304367,
  0.049860004,
  -0.06350248,
  0.02514655,
  0.029342154,
  -0.008086687,
  -0.05895499,
  0.018555393,
  -0.00033497153,
  0.0440403,
  -0.01921857,
  -0.029802317,
  0.07535844,
  0.01758093,
  -0.021546451,
  -0.006638527,
  -0.026

In [29]:
index=pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace="ns1",
    
)

{'upserted_count': 20}

In [31]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}